In [7]:
# 79-character line limit
######### ######### ######### ######### ######### ######### ######### #########
%reset -f

import numpy as np # The NumPy library
#import math # The math module, np includes it
from scipy.integrate import quad # Method for integration in scipy.integrate sub-package


cells_num = 32

# The sizes are for UQs for number of cells: 4, 8, 16, 32, 64, 128, 256.

# The opitimized spaces for the Guassian distribution in a uniform quantizer(UQ).
all_size_gaus = np.array([0.9957, 0.5860, 0.3352, 0.1881, 0.1041, 0.0569, 0.0308])

# The opitimized spaces for the Laplacian distribution in a uniform quantizer(UQ).
all_size_laplacian = np.array(
                   [1.414, 1.0873, 0.8707, 0.7309, 0.6334, 0.5613, 0.5055])


'''The opitimized spaces for a Mixed Gaussian distribution in 
a uniform quantizer(UQ). This distribution is not symmetric and the values are
for for cells number 2, 4, 8, 16, 32, 64, 128, 256'''
all_boudary1s = [-0.05, -2.02, -3.17, -3.93, -4.49, -5.08, -5.16, -5.50]
all_deltas = [4, 1.81, 1, 0.54, 0.29, 0.16, 0.08, 0.05]

'''To change the symmetric pdf, two parts of code must be changed include the 
cell_size and pdf'''

cell_size = all_size_gaus[int(np.log2(cells_num))-2]
# cell_size = all_size_laplacian[int(np.log2(cells_num))-2]

# np.arange has rounding error issue.
# boundaries_symm = np.arange(-(cells_num/2-1) * cell_size, # For symmetric distributions
#                              (cells_num/2) * cell_size,
#                             cell_size)

# Boundaries for the symmetric distributions are computed.
boundaries_symm = np.linspace(-(cells_num/2-1) * cell_size, # For symmetric distributions
                               (cells_num/2-1) * cell_size,
                              cells_num-1)

# Boundaries for the asymmetric distributions are computed.
boundary1 = all_boudary1s[int(np.log2(cells_num))-1]
delta = all_deltas[int(np.log2(cells_num))-1]

boundaries_asymm = []
for i in range (0,cells_num-1):
        boundary = boundary1 + (i) * delta
        boundaries_asymm = np.append(boundaries_asymm, boundary)
        
'''To change the symmetric pdf to asymetric pdf, two parts of code 
must be changed include the boundaries and pdf'''        

# boundaries = boundaries_symm # For symmetric distributions
boundaries = boundaries_asymm # For asymetric distributions
        

n_inf = float("-inf")
p_inf = float("inf")

boundaries = np.insert(boundaries, 0, n_inf)
boundaries = np.append(boundaries, p_inf)

def pdf(x): # Defining the distribution
#     gaus_std = 1
#     gaus_mean = 0
#     pdf = 1/(gaus_std*np.sqrt(2*np.pi)) * \
#                   np.exp(-0.5*((x-gaus_mean)/gaus_std)**2) # Gaussian pdf
    
#     div = 1
#     pdf = 1/(2*div)*np.exp(-abs(x)/div) # Laplacian pdf

    Mean1 = -2
    Mean2 = 2
    stdev1 = 1
    stdev2 = 1
    alpha = 0.7
    pdf = (alpha)*(1/np.sqrt(2*np.pi*stdev1)*np.exp(-(x-Mean1)**2/(2*stdev1**2)))\
        + (1-alpha)*(1/np.sqrt(2*np.pi*stdev2)*np.exp(-(x-Mean2)**2/(2*stdev2**2)))
    
    return pdf

def xpdf(x):
    xpdf = x * pdf(x)
    return xpdf

def x2pdf (x):
    x2pdf = x * xpdf(x)
    return x2pdf


prbs = []
xprbs = []
x2prbs = []
cell_reps = []
for i in range (0, cells_num):
    cell_prb, integ_err = quad(pdf, boundaries[i], boundaries[i+1])
    prbs = np.append(prbs, cell_prb)
    
    cell_xprb, integ_err = quad(xpdf, boundaries[i], boundaries[i+1])
    xprbs = np.append(xprbs, cell_xprb)
    
    cell_x2prb, integ_err = quad(x2pdf, boundaries[i], boundaries[i+1])
    x2prbs = np.append(x2prbs, cell_x2prb)
    
    cell_rep = cell_xprb / cell_prb
    cell_reps = np.append(cell_reps, cell_rep)
    
    
# Function definitions
def relu(x):
    return np.maximum(0,x)

def lrelu(x):
    return np.where(x > 0, x, x * 0.1)

def sigmoid(x):
    return 1/(1 + np.exp(-x))

In [8]:
# Weights and biases initialization

l1_size = 32

# W1 = np.random.randn(l1_size, cells_num)
# B1 = np.zeros(l1_size,)
# W2 = np.random.randn(cells_num, l1_size)


# "He Initialization" technique
W1 = np.random.randn(l1_size, cells_num) * np.sqrt(2/cells_num)
B1 = np.zeros(l1_size,)
W2 = np.random.randn(cells_num, l1_size) * np.sqrt(2/l1_size)

In [12]:
# First feed forward

l1_outs = relu(np.dot(W1, cell_reps)+B1)
l2_outs = np.dot(W2, l1_outs)
l2_outs = np.round(l2_outs, 16)

# Indexes based on the final layer outputs.
sorted_outs = np.sort(l2_outs)
indexes = []
for i in range(0, cells_num):
    index = np.where(sorted_outs == l2_outs[i])
    indexes = np.append(indexes, index)
    
yijkls = [] # Yijkls based on the assigned indexes
dijkls = []
for i in range(0, int(cells_num/4)):
    j = i + cells_num/4
    k = i + 2 * cells_num/4
    l = i + 3 * cells_num/4

    celli = np.where(indexes == i)
    cellj = np.where(indexes == j)
    cellk = np.where(indexes == k)
    celll = np.where(indexes == l)

    yijkl = (xprbs[celli] + xprbs[cellj] + xprbs[cellk] + xprbs[celll])\
           /(prbs[celli] + prbs[cellj] + prbs[cellk] + prbs[celll])
    yijkls = np.append(yijkls, yijkl)

    dijkl = x2prbs[celli] + yijkl**2 * prbs[celli] - yijkl *2 * xprbs[celli]\
          + x2prbs[cellj] + yijkl**2 * prbs[cellj] - yijkl *2 * xprbs[cellj]\
          + x2prbs[cellk] + yijkl**2 * prbs[cellk] - yijkl *2 * xprbs[cellk]\
          + x2prbs[celll] + yijkl**2 * prbs[celll] - yijkl *2 * xprbs[celll]
    dijkls = np.append(dijkls, dijkl)

distortion = np.round(sum(dijkls), 16) # The Eve's distortion based on the assigned indexes

print('indexes for cells', indexes)
print('\n', distortion)

quads = []
for i in range(int(cells_num/4)):
    quad_cell1 = np.where(indexes==i)
    quad_cell2 = np.where(indexes==i+cells_num/4)
    quad_cell3 = np.where(indexes==i+2*cells_num/4)
    quad_cell4 = np.where(indexes==i+3*cells_num/4)
    quad = [quad_cell1, quad_cell2, quad_cell3, quad_cell4]
    quads = np.append(quads, quad)

quads = np.transpose(np.reshape(quads,[int(cells_num/4), 4]))
print('\n','quad_cells','\n', quads)

tW1 = W1.copy()
tB1 = B1.copy()
tW2 = W2.copy()


past_velocity1 = 0 # Past velocity for momentum
past_velocity_b1 = 0
past_velocity2 = 0


m_adam1 = 0 # For Adam
v_adam1 = 0
m_adam_b1 = 0
v_adam_b1 = 0
m_adam2 = 0
v_adam2 = 0


Gt1 = 0 # For Adagrad
Gtb1 = 0
Gt2 = 0
adagrad_eps = 0.0001


past1 = 0.001 # For conjugate gradient
pastp1 = 0.001
pastb1 = 0.001
pastpb1 = 0.001
    
past2 = 0.001
pastp2 = 0.001

indexes for cells [22.  8. 12.  7. 14.  0.  4.  9. 19. 21. 13.  1. 26.  2.  6. 31. 29. 20.
 18.  3.  5. 25. 30. 27. 10. 17. 11. 16. 24. 15. 23. 28.]

 4.019971616023234

 quad_cells 
 [[ 5. 11. 13. 19.  6. 20. 14.  3.]
 [ 1.  7. 24. 26.  2. 10.  4. 29.]
 [27. 25. 18.  8. 17.  9.  0. 30.]
 [28. 21. 12. 23. 31. 16. 22. 15.]]


In [ ]:
# Training the network
'''This part provides the derivative of the distortion w.r.t. the weights
for the back propagation. The prefix 'b' shows that the variables are
calculated temporarily for the gradient descent'''

learning_rate = 0.1
epsilon = 0.1
 
momentum = 0.55

beta_1 = 0.9 # Parameters of Adam
beta_2 = 0.999


iterations = 1000

for ii in range (1, iterations):
    l1_gradients = [] # Gradients of layer 1 weights
#     learning_rate = 100/ii
    for i in range (0, l1_size):
        for j in range (0, cells_num):
            bW1 = tW1.copy()
            bW1[i][j] = tW1 [i][j] + epsilon
            b_l1_outs = relu(np.dot(bW1, cell_reps)+tB1)
            b_l2_outs = np.dot(tW2, b_l1_outs)
            b_l2_outs = np.round(b_l2_outs, 16)
            
            for p in range(0, cells_num-1):
                for q in range (p+1, cells_num):
                    if b_l2_outs[p] == b_l2_outs[q]:
                        b_l2_outs[q] += np.random.rand(1,1)-0.5
            
            b_sorted_outs = np.sort(b_l2_outs)
            b_indexes = []
            for k in range(0, cells_num):
                b_index = np.where(b_sorted_outs == b_l2_outs[k])
                b_indexes = np.append(b_indexes, b_index)

            byijkls = [] # Yijkls based on the assigned indexes
            bdijkls = []
            for i in range(0, int(cells_num/4)):
                j = i + cells_num/4
                k = i + 2 * cells_num/4
                l = i + 3 * cells_num/4

                bcelli = np.where(b_indexes == i)
                bcellj = np.where(b_indexes == j)
                bcellk = np.where(b_indexes == k)
                bcelll = np.where(b_indexes == l)

                byijkl = (xprbs[bcelli] + xprbs[bcellj] + xprbs[bcellk] + xprbs[bcelll])\
                        /(prbs[bcelli] + prbs[bcellj] + prbs[bcellk] + prbs[bcelll])
                byijkls = np.append(byijkls, byijkl)

                bdijkl = x2prbs[bcelli] + byijkl**2 * prbs[bcelli] - byijkl *2 * xprbs[bcelli]\
                       + x2prbs[bcellj] + byijkl**2 * prbs[bcellj] - byijkl *2 * xprbs[bcellj]\
                       + x2prbs[bcellk] + byijkl**2 * prbs[bcellk] - byijkl *2 * xprbs[bcellk]\
                       + x2prbs[bcelll] + byijkl**2 * prbs[bcelll] - byijkl *2 * xprbs[bcelll]
                bdijkls = np.append(bdijkls, bdijkl)

            b_distortion = np.round(sum(bdijkls), 16)
                        
            l1_gradient = (b_distortion - distortion)/epsilon
            l1_gradients = np.append(l1_gradients, l1_gradient)
            
    l1_gradients = np.reshape(l1_gradients, [l1_size, cells_num])
        
    
    
    
    l1_biases_gradients = [] # Gradients of the layer 1 biases
    for i in range (0, l1_size):
        bB1 = tB1.copy()
        bB1 [i] = tB1 [i] + epsilon
        b_l1_outs = relu(np.dot(tW1, cell_reps)+bB1)
        b_l2_outs = np.dot(tW2, b_l1_outs)
        b_l2_outs = np.round(b_l2_outs, 16)
        
        for p in range(0, cells_num-1):
            for q in range (p+1, cells_num):
                if b_l2_outs[p] == b_l2_outs[q]:
                    b_l2_outs[q] += np.random.rand(1,1)-0.5

        b_sorted_outs = np.sort(b_l2_outs)
        b_indexes = []
        
        for k in range(0, cells_num):
            b_index = np.where(b_sorted_outs == b_l2_outs[k])
            b_indexes = np.append(b_indexes, b_index)

            byijkls = [] # Yijkls based on the assigned indexes
            bdijkls = []
            for i in range(0, int(cells_num/4)):
                j = i + cells_num/4
                k = i + 2 * cells_num/4
                l = i + 3 * cells_num/4

                bcelli = np.where(b_indexes == i)
                bcellj = np.where(b_indexes == j)
                bcellk = np.where(b_indexes == k)
                bcelll = np.where(b_indexes == l)

                byijkl = (xprbs[bcelli] + xprbs[bcellj] + xprbs[bcellk] + xprbs[bcelll])\
                        /(prbs[bcelli] + prbs[bcellj] + prbs[bcellk] + prbs[bcelll])
                byijkls = np.append(byijkls, byijkl)

                bdijkl = x2prbs[bcelli] + byijkl**2 * prbs[bcelli] - byijkl *2 * xprbs[bcelli]\
                       + x2prbs[bcellj] + byijkl**2 * prbs[bcellj] - byijkl *2 * xprbs[bcellj]\
                       + x2prbs[bcellk] + byijkl**2 * prbs[bcellk] - byijkl *2 * xprbs[bcellk]\
                       + x2prbs[bcelll] + byijkl**2 * prbs[bcelll] - byijkl *2 * xprbs[bcelll]
                bdijkls = np.append(bdijkls, bdijkl)

            b_distortion = np.round(sum(bdijkls), 16)
            
        l1_biases_gradient = (b_distortion - distortion)/epsilon
        l1_biases_gradients = np.append(l1_biases_gradients, l1_biases_gradient)
    
           
        
        
    l2_gradients = [] # Gradients of layer 2 weights
    for i in range (0, cells_num):
        for j in range (0, l1_size):
            bW2 = tW2.copy()
            bW2[i][j] = tW2[i][j] + epsilon
            b_l1_outs = relu(np.dot(tW1, cell_reps)+tB1)
            b_l2_outs = np.dot(bW2, b_l1_outs)
            b_l2_outs = np.round(b_l2_outs, 16)
            
            for p in range(0, cells_num-1):
                for q in range (p+1, cells_num):
                    if b_l2_outs[p] == b_l2_outs[q]:
                        b_l2_outs[q] += np.random.rand(1,1)-0.5


            b_sorted_outs = np.sort(b_l2_outs)
            b_indexes = []
            for k in range(0, cells_num):
                b_index = np.where(b_sorted_outs == b_l2_outs[k])
                b_indexes = np.append(b_indexes, b_index)

            byijkls = [] # Yijkls based on the assigned indexes
            bdijkls = []
            for i in range(0, int(cells_num/4)):
                j = i + cells_num/4
                k = i + 2 * cells_num/4
                l = i + 3 * cells_num/4

                bcelli = np.where(b_indexes == i)
                bcellj = np.where(b_indexes == j)
                bcellk = np.where(b_indexes == k)
                bcelll = np.where(b_indexes == l)

                byijkl = (xprbs[bcelli] + xprbs[bcellj] + xprbs[bcellk] + xprbs[bcelll])\
                        /(prbs[bcelli] + prbs[bcellj] + prbs[bcellk] + prbs[bcelll])
                byijkls = np.append(byijkls, byijkl)

                bdijkl = x2prbs[bcelli] + byijkl**2 * prbs[bcelli] - byijkl *2 * xprbs[bcelli]\
                       + x2prbs[bcellj] + byijkl**2 * prbs[bcellj] - byijkl *2 * xprbs[bcellj]\
                       + x2prbs[bcellk] + byijkl**2 * prbs[bcellk] - byijkl *2 * xprbs[bcellk]\
                       + x2prbs[bcelll] + byijkl**2 * prbs[bcelll] - byijkl *2 * xprbs[bcelll]
                bdijkls = np.append(bdijkls, bdijkl)

            b_distortion = np.round(sum(bdijkls), 16)
                        
            l2_gradient = (b_distortion - distortion)/epsilon
            l2_gradients = np.append(l2_gradients, l2_gradient)
            
    l2_gradients = np.reshape(l2_gradients, [cells_num, l1_size])
    

# # Now, we update the weights.   
#     tW1 = tW1 + learning_rate * l1_gradients
#     tB1 = tB1 + learning_rate * l1_biases_gradients
#     tW2 = tW2 + learning_rate * l2_gradients
    
    
    
    
    # We update the weights with momentum.
    velocity1 = past_velocity1 * momentum + learning_rate *  l1_gradients
    tW1 = tW1 - momentum * velocity1 + learning_rate * l1_gradients
    past_velocity1 = velocity1 
    
    velocity_b1 = past_velocity_b1 * momentum + learning_rate *  l1_biases_gradients
    tB1 = tB1 - momentum * velocity_b1 + learning_rate * l1_biases_gradients
    past_velocity_b1 = velocity_b1
    
    velocity2 = past_velocity2 * momentum + learning_rate *  l2_gradients
    tW2 = tW2 - momentum * velocity2 + learning_rate * l2_gradients
    past_velocity2 = velocity2
    
    
#     # momentum way 2
#     velocity1 = learning_rate * l1_gradients + momentum * past_velocity1
#     tW1 = tW1 + velocity1
#     past_velocity1 = velocity1
    
#     velocity2 = learning_rate * l2_gradients + momentum * past_velocity2
#     tW2 = tW2 + velocity2
#     past_velocity2 = velocity2
    
#     velocity_b1 = learning_rate * l1_biases_gradients + momentum * past_velocity_b1
#     tB1 = tB1 + velocity_b1
#     past_velocity_b1 = velocity_b1




# # Here, we implement the Adam algorithm.

#     m_adam1 = beta_1 * m_adam1 + (1 - beta_1) * l1_gradients
#     v_adam1 = beta_2 * v_adam1 + (1 - beta_2) * np.power(l1_gradients, 2)
#     m_hat1 = m_adam1 / (1 - np.power(beta_1, ii))
#     v_hat1 = v_adam1 / (1 - np.power(beta_2, ii))
#     tW1 = tW1 + learning_rate * m_hat1 / (np.sqrt(v_hat1) + 0.001)
    
#     m_adam_b1 = beta_1 * m_adam_b1 + (1 - beta_1) * l1_biases_gradients
#     v_adam_b1 = beta_2 * v_adam_b1 + (1 - beta_2) * np.power(l1_biases_gradients, 2)
#     m_hat_b1 = m_adam_b1 / (1 - np.power(beta_1, ii))
#     v_hat_b1 = v_adam_b1 / (1 - np.power(beta_2, ii))
#     tB1 = tB1 + learning_rate * m_hat_b1 / (np.sqrt(v_hat_b1) + 0.001)
    
#     m_adam2 = beta_1 * m_adam2 + (1 - beta_1) * l2_gradients
#     v_adam2 = beta_2 * v_adam2 + (1 - beta_2) * np.power(l2_gradients, 2)
#     m_hat2 = m_adam2 / (1 - np.power(beta_1, ii))
#     v_hat2 = v_adam2 / (1 - np.power(beta_2, ii))
#     tW2 = tW2 + learning_rate * m_hat2 / (np.sqrt(v_hat2) + 0.001)
    
    
    
    
# # Adagrad
#     Gt1 += np.dot(l1_gradients, np.transpose(l1_gradients))
#     Gt1_diag_elements = np.diag(Gt1)
#     Gt1_diag = np.zeros(np.shape(Gt1))
#     np.fill_diagonal(Gt1_diag,Gt1_diag_elements)
#     lr1_denominator = np.sqrt(adagrad_eps + Gt1_diag)
    
# #     Gtb1 += np.dot(l1_biases_gradients, np.transpose(l1_biases_gradients))
# #     Gtb1_diag_elements = np.diag(Gtb1)
# #     Gtb1_diag = np.zeros(np.shape(Gtb1))
# #     np.fill_diagonal(Gtb1_diag,Gtb1_diag_elements)
# #     lrb1_denominator = np.sqrt(adagrad_eps + Gtb1_diag)
    
#     Gt2 += np.dot(l2_gradients, np.transpose(l2_gradients))
#     Gt2_diag_elements = np.diag(Gt2)
#     Gt2_diag = np.zeros(np.shape(Gt2))
#     np.fill_diagonal(Gt2_diag,Gt2_diag_elements)
#     lr2_denominator = np.sqrt(adagrad_eps + Gt2_diag)
    
        
#     tW1 = tW1 + learning_rate/lr1_denominator * l1_gradients
#     tB1 = tB1 + learning_rate * l1_biases_gradients
#     tW2 = tW2 + learning_rate/lr2_denominator * l2_gradients
    
    
       
    
# # Conjugate gradient
#     beta1 = np.dot(np.transpose(l1_gradients - past1),l1_gradients)/\
#             (np.dot(np.transpose(past1),past1)+0.001)
#     p1 = -l1_gradients + beta1 * pastp1
#     past1 = l1_gradients
#     pastp1 = p1
    
#     betab1 = np.dot(np.transpose(l1_biases_gradients - pastb1),l1_biases_gradients)/\
#              (np.dot(np.transpose(pastb1),pastb1)+0.001)
#     pb1 = -l1_biases_gradients + betab1 * pastpb1
#     pastb1 = l1_biases_gradients
#     pastpb1 = pb1
    
#     beta2 = np.dot(np.transpose(l2_gradients - past2),l2_gradients)/\
#             (np.dot(np.transpose(past2),past2)+0.001)
#     p2 = -l2_gradients + beta2 * pastp2
#     past2 = l2_gradients
#     pastp2 = p2
    
          
#     tW1 = tW1 + learning_rate * (l1_gradients + beta1 * pastp1)
#     tB1 = tB1 + learning_rate * (l1_biases_gradients + betab1 * pastpb1)
#     tW2 = tW2 + learning_rate * (l2_gradients + beta2 * pastp2)    

   
    
    
    
    
    
    l1_outs = relu(np.dot(tW1,cell_reps)+tB1)
    l2_outs = np.dot(tW2, l1_outs)
    l2_outs = np.round(l2_outs, 16)
    
    for p in range(0, cells_num-1):
        for q in range (p+1, cells_num):
            if l2_outs[p] == l2_outs[q]:
                l2_outs[q] += np.random.rand(1,1)-0.5
   

    # We assigned the indexes based on the final layer outputs.
    sorted_outs = np.sort(l2_outs)
    indexes = []
    for i in range(0, cells_num):
        index = np.where(sorted_outs == l2_outs[i])
        indexes = np.append(indexes, index)

    #print(indexes)
    
    yijkls = [] # Yijkls based on the assigned indexes
    dijkls = []
    for i in range(0, int(cells_num/4)):
        j = i + cells_num/4
        k = i + 2 * cells_num/4
        l = i + 3 * cells_num/4

        celli = np.where(indexes == i)
        cellj = np.where(indexes == j)
        cellk = np.where(indexes == k)
        celll = np.where(indexes == l)

        yijkl = (xprbs[celli] + xprbs[cellj] + xprbs[cellk] + xprbs[celll])\
               /(prbs[celli] + prbs[cellj] + prbs[cellk] + prbs[celll])
        yijkls = np.append(yijkls, yijkl)

        dijkl = x2prbs[celli] + yijkl**2 * prbs[celli] - yijkl *2 * xprbs[celli]\
              + x2prbs[cellj] + yijkl**2 * prbs[cellj] - yijkl *2 * xprbs[cellj]\
              + x2prbs[cellk] + yijkl**2 * prbs[cellk] - yijkl *2 * xprbs[cellk]\
              + x2prbs[celll] + yijkl**2 * prbs[celll] - yijkl *2 * xprbs[celll]
        dijkls = np.append(dijkls, dijkl)

    distortion = np.round(sum(dijkls), 16) # The Eve's distortion based on the assigned indexes

    
    print(distortion)

print(indexes)
quads = []
for i in range(int(cells_num/4)):
    quad_cell1 = np.where(indexes==i)
    quad_cell2 = np.where(indexes==i+cells_num/4)
    quad_cell3 = np.where(indexes==i+2*cells_num/4)
    quad_cell4 = np.where(indexes==i+3*cells_num/4)
    quad = [quad_cell1, quad_cell2, quad_cell3, quad_cell4]
    quads = np.append(quads, quad)

quads = np.transpose(np.reshape(quads,[int(cells_num/4), 4]))
print('\n', quads)

In [14]:
distortion

2.987011290272516